Alpha=1.5(CNNRNN is good)

In [1]:
import sys
sys.path.append("..")

In [ ]:
import torch
import numpy as np
from torch import nn, optim
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
from tqdm import tqdm
from scienceplots import styles
import os
from pathlib import Path
from datetime import datetime
import import_ipynb
from modelsAndFunctions.models_CNN_RNN import model

In [6]:
# 初始化设置
plt.style.use(["science", "ieee", "no-latex"])
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [7]:
class Args:
    def __init__(self):
        self.learning_rate = 0.0001
        self.max_epoch = 100  # 与图中7000次迭代对应
        self.batch_size = 64
        self.session_name = datetime.now().strftime('%b%d_%H%M%S')
        self.test_checkpoint = None
        self.nonlinearity = "tanh"

In [ ]:
def load_normalized_data():
    """加载已预处理的标准化数据"""
    # 假设数据已经预处理并保存在指定路径
    data_path = Path(r".\3DTensor-basedDL4OptionPricing\data_ipynb\torch-data-ipynb")
    
    train_input = torch.load(f"{data_path}\\train_input(V).pt").float().cuda()
    train_label = torch.load(f"{data_path}\\train_label(V).pt").float().cuda()
    
    val_input = torch.load(f"{data_path}\\val_input.pt").float().cuda()  # 新增验证集
    val_label = torch.load(f"{data_path}\\val_label.pt").float().cuda()
    
    test_input = torch.load(f"{data_path}\\test_input(alpha=1.5)(V).pt").float().cuda()
    test_label = torch.load(f"{data_path}\\test_label(alpha=1.5)(V).pt").float().cuda()
    
    return (train_input, train_label), (val_input, val_label), (test_input, test_label)

In [9]:
def create_dataloaders(args):
    """创建数据加载器"""
    (train_input, train_label), (val_input, val_label), _ = load_normalized_data()
    
    train_loader = DataLoader(
        TensorDataset(train_input, train_label),
        batch_size=args.batch_size,
        shuffle=True
    )
    
    val_loader = DataLoader(
        TensorDataset(val_input, val_label),
        batch_size=args.batch_size
    )
    
    return train_loader, val_loader  # 添加这行返回语句

In [10]:
def initialize_model(args):
    """初始化模型和优化器"""
    net = model(  
        resolution_ratio=4,
        nonlinearity=args.nonlinearity,
        in_dim=5,
        in_channels=3,
        out_dim=1,
        seq_len=10
    ).cuda()
    
    optimizer = optim.Adam(
        net.parameters(),
        lr=args.learning_rate,
        amsgrad=True
    )
    
    return net, optimizer


In [11]:
def train_and_validate(net, optimizer, train_loader, val_loader, args):
    """完整的训练验证流程"""
    criterion = nn.MSELoss()
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=10)
    
    train_losses, val_losses = [], []
    best_val_loss = float('inf')

    for epoch in tqdm(range(args.max_epoch), desc="Training"):
        # 训练阶段
        net.train()
        epoch_train_loss = []
        for x, y in train_loader:
            optimizer.zero_grad()
            y_pred = net(x)
            loss = criterion(y_pred, y.unsqueeze(1))
            loss.backward()
            optimizer.step()
            epoch_train_loss.append(loss.item())
        
        train_losses.append(np.mean(epoch_train_loss))

        # 验证阶段
        net.eval()
        epoch_val_loss = []
        with torch.no_grad():
            for x, y in val_loader:
                y_pred = net(x)
                loss = criterion(y_pred, y.unsqueeze(1))
                epoch_val_loss.append(loss.item())
        
        val_loss = np.mean(epoch_val_loss)
        val_losses.append(val_loss)
        scheduler.step(val_loss)

        # 保存最佳模型
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(net.state_dict(), f"checkpoints/{args.session_name}_best.pth")

    # 保存最终模型
    torch.save(net.state_dict(), f"checkpoints/{args.session_name}_final.pth")
    return train_losses, val_losses

In [ ]:
def plot_ieee_error_curves(train_losses, val_losses, session_name):
    """IEEE标准误差曲线绘制"""
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(3.5, 5), sharex=True)
    
    # 上方子图：绝对误差（元）
    ax1.plot(train_losses, color='#e377c2', linewidth=1, label="Training")
    ax1.plot(val_losses, color='#2ca02c', linestyle=':', linewidth=1.5, label="Validation")
    ax1.set_ylabel('Absolute Error ($)', fontsize=8)
    ax1.legend(frameon=False, fontsize=7)
    
    # 下方子图：MAE（百分比）
    '''ax2.plot(np.array(train_losses)*100, color='#e377c2', linewidth=1)
    ax2.plot(np.array(val_losses)*100, color='#2ca02c', linestyle=':', linewidth=1.5)
    ax2.set_xlabel('Iterations (in Thousands)', fontsize=8)
    ax2.set_ylabel('Mean Absolute Error', fontsize=8)'''
        # 上方子图：绝对误差（元）
    ax1.plot(train_losses, color='#e377c2', linewidth=1, label="Training")
    ax1.plot(val_losses, color='#2ca02c', linestyle=':', linewidth=1.5, label="Validation")
    ax1.set_ylabel('Absolute Error ($)', fontsize=8)
    ax1.legend(frameon=False, fontsize=7)
    
    # 设置x轴刻度
    '''xticks = np.linspace(0, len(train_losses), 5)
    ax2.set_xticks(xticks)
    ax2.set_xticklabels([f"{int(x/1000)}k" for x in xticks], fontsize=7)'''
        # 下方子图：MAE（百分比）
    mae_train = np.array(train_losses) * 100  # 转为百分比
    mae_val = np.array(val_losses) * 100
    ax2.plot(mae_train, color='#e377c2', linewidth=1)
    ax2.plot(mae_val, color='#2ca02c', linestyle=':', linewidth=1.5)
    ax2.set_xlabel('Iterations (in Thousands)', fontsize=8)
    ax2.set_ylabel('Mean Absolute Error', fontsize=8)
    
    '''plt.tight_layout()
    os.makedirs("figures", exist_ok=True)
    plt.savefig(f"figures/{session_name}_error_curves.pdf", bbox_inches='tight', dpi=300)
    plt.show()'''
    
     # 设置y轴刻度间距为0.1
    max_mae = max(max(mae_train), max(mae_val))
    mae_ticks = np.arange(0, max_mae + 0.1, 0.1)  # 每0.1一格
    ax2.set_yticks(mae_ticks)
    ax2.set_yticklabels([f"{tick:.1f}" for tick in mae_ticks], fontsize=7)  # 保留一位小数
    
    # 设置x轴刻度（保持不变）
    xticks = np.linspace(0, len(train_losses), 5)
    ax2.set_xticks(xticks)
    ax2.set_xticklabels([f"{int(x/1000)}k" for x in xticks], fontsize=7)
    
    plt.tight_layout()
    os.makedirs("figures", exist_ok=True)
    plt.savefig(f"figures/{session_name}_error_curves.pdf", bbox_inches='tight', dpi=300)
    plt.show()


In [ ]:
if __name__ == "__main__":
    # 初始化配置
    args = Args()
    os.makedirs("checkpoints", exist_ok=True)
    
    # 数据准备
    train_loader, val_loader = create_dataloaders(args)
    
    # 模型初始化
    net, optimizer = initialize_model(args)
    
    # 训练验证流程
    train_losses, val_losses = train_and_validate(
        net, optimizer, train_loader, val_loader, args
    )
    
    # 绘制专业图表
    plot_ieee_error_curves(train_losses, val_losses, args.session_name)